In [1]:
#import dependencies
import pandas as pd
import numpy as np

In [2]:
#set path for csv file
csvpath = "Resources/purchase_data.csv"
#import the purchase_data.csv file as a DataFrame
purchase_data_df = pd.read_csv(csvpath)
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#Player Count
#calculate total players
total_players = len(purchase_data_df["SN"].unique())
total_players

576

In [4]:
#place Player Count into a summary DataFrame
player_count_df = pd.DataFrame({"Total Players": [total_players]})
player_count_df

,Total Players
0,576


In [5]:
#Purchasing Analysis (Total); PAT

#need to find: number of items, average price, total # of purchases, total revenue
#be careful of ordering

#calculate the number of unique items
items = len(purchase_data_df["Item ID"].unique())

#calculate the total revenue
total_revenue = sum(purchase_data_df["Price"])

#calculate the total number of purchases
total_number_of_purchases = len(purchase_data_df["Price"])

#calculate the average price 
average_price = total_revenue / total_number_of_purchases

In [6]:
# place all of the Purchasing Analysis data found into a summary DataFrame
PAT_df = pd.DataFrame({"Number of Unique Items": [items],
                                                  "Average Price": [average_price],
                                                  "Number of Purchases": [total_number_of_purchases],
                                                  "Total Revenue": [total_revenue]                                                  
                                                 })
PAT_df.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.050987,780,2379.77


In [7]:
#now format Average Price and Total Revenue in the summary table
PAT_df["Average Price"] = PAT_df["Average Price"].map("${:,.2f}".format)
PAT_df["Total Revenue"] = PAT_df["Total Revenue"].map("${:,.2f}".format)

PAT_df.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [8]:
#Gender Demographics
#find each gender's total count and percentage makeup of all players

#need to drop the duplicate players in order to find the numbers we're after
player_demo_df = purchase_data_df.loc[:, ["Gender", "SN", "Age"]]
player_demo_df = player_demo_df.drop_duplicates()

#generate totals for each gender
gender_total = player_demo_df["Gender"].value_counts()

#generate percents for each gender
gender_percent = gender_total / total_players 

In [9]:
#put gender demographics into a summary table
gender_demo_df = pd.DataFrame({"Total Count": gender_total, "Percentage of Players": gender_percent})
gender_demo_df

,Total Count,Percentage of Players
Male,484,0.840278
Female,81,0.140625
Other / Non-Disclosed,11,0.019097


In [10]:
#format the percentage of players
gender_demo_df["Percentage of Players"] = gender_demo_df["Percentage of Players"].map("{:,.2%}".format)
gender_demo_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [11]:
#Purchasing Analysis (Gender)

#create a dataframe, grouping by gender
gender_groupby = purchase_data_df.groupby(["Gender"])
gender_groupby.count()

,Purchase ID,SN,Age,Item ID,Item Name,Price
Gender,,,,,,
Female,113,113,113,113,113,113
Male,652,652,652,652,652,652
Other / Non-Disclosed,15,15,15,15,15,15


In [12]:
type(gender_groupby)

pandas.core.groupby.generic.DataFrameGroupBy

In [13]:
purchase_count = gender_groupby["Purchase ID"].count()
purchase_count
print(purchase_count)

total_purchased = gender_groupby["Price"].sum()
total_purchased
print(total_purchased)

ave_price = total_purchased / purchase_count
ave_price
print(ave_price)

ave_total_purch_per_person = total_purchased / gender_total
ave_total_purch_per_person
print(ave_total_purch_per_person)

Gender
Female                   113
Male                     652
Other / Non-Disclosed     15
Name: Purchase ID, dtype: int64
Gender
Female                    361.94
Male                     1967.64
Other / Non-Disclosed      50.19
Name: Price, dtype: float64
Gender
Female                   3.203009
Male                     3.017853
Other / Non-Disclosed    3.346000
dtype: float64
Female                   4.468395
Male                     4.065372
Other / Non-Disclosed    4.562727
dtype: float64


In [14]:
dom_df = pd.DataFrame({"Purchase Count": purchase_count,
                       "Average Purchase Price": ave_price,
                       "Total Purchase Value": total_purchased,
                       "Avg Total Purchase per Person": ave_total_purch_per_person})
dom_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


In [15]:
dom_df["Average Purchase Price"] = dom_df["Average Purchase Price"].map("${:,.2f}".format)
dom_df["Total Purchase Value"] = dom_df["Total Purchase Value"].map("${:,.2f}".format)
dom_df["Avg Total Purchase per Person"] = dom_df["Avg Total Purchase per Person"].map("${:,.2f}".format)
dom_df                                                         

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [16]:
#Age Demographics

#figure out the minimum and maximum ages for players
print(player_demo_df["Age"].max())
print(player_demo_df["Age"].min())

45
7


In [17]:
#establish bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]

group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [18]:
pd.cut(player_demo_df["Age"], bins, labels=group_labels).head()
#series

0    20-24
1      40+
2    20-24
3    20-24
4    20-24
Name: Age, dtype: category
Categories (8, object): [<10 < 10-14 < 15-19 < 20-24 < 25-29 < 30-34 < 35-39 < 40+]

In [19]:
player_demo_df["Age Group"] = pd.cut(player_demo_df["Age"], bins, labels=group_labels)
player_demo_df
#DF

,Gender,SN,Age,Age Group
0,Male,Lisim78,20,20-24
1,Male,Lisovynya38,40,40+
2,Male,Ithergue48,24,20-24
3,Male,Chamassasya86,24,20-24
4,Male,Iskosia90,23,20-24
...,...,...,...,...
773,Male,Hala31,21,20-24
774,Male,Jiskjask80,11,10-14
775,Female,Aethedru70,21,20-24
777,Male,Yathecal72,20,20-24


In [20]:
age_counts = player_demo_df["Age Group"].value_counts()
age_counts

20-24    258
15-19    107
25-29     77
30-34     52
35-39     31
10-14     22
<10       17
40+       12
Name: Age Group, dtype: int64

In [21]:
total_gender = player_demo_df["Gender"].value_counts()
total_gender

Male                     484
Female                    81
Other / Non-Disclosed     11
Name: Gender, dtype: int64

In [22]:
age_percent_makeup = age_counts / total_players
age_percent_makeup

20-24    0.447917
15-19    0.185764
25-29    0.133681
30-34    0.090278
35-39    0.053819
10-14    0.038194
<10      0.029514
40+      0.020833
Name: Age Group, dtype: float64

In [23]:
age_demographics_table_df = pd.DataFrame({"Total Count": age_counts,
                                          "Percentage of Players": age_percent_makeup}).sort_index()
age_demographics_table_df

,Total Count,Percentage of Players
<10,17,0.029514
10-14,22,0.038194
15-19,107,0.185764
20-24,258,0.447917
25-29,77,0.133681
30-34,52,0.090278
35-39,31,0.053819
40+,12,0.020833


In [24]:
age_demographics_table_df["Percentage of Players"] = age_demographics_table_df["Percentage of Players"].map("{:,.2%}".format)
age_demographics_table_df

,Total Count,Percentage of Players
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [25]:
#Purchasing Analysis (Age)

pd.cut(purchase_data_df["Age"], bins, labels=group_labels).head()

0    20-24
1      40+
2    20-24
3    20-24
4    20-24
Name: Age, dtype: category
Categories (8, object): [<10 < 10-14 < 15-19 < 20-24 < 25-29 < 30-34 < 35-39 < 40+]

In [26]:
purchase_data_df["Age Group"] = pd.cut(purchase_data_df["Age"], bins, labels=group_labels)
purchase_data_df

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Group
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24
...,...,...,...,...,...,...,...,...
775,775,Aethedru70,21,Female,60,Wolf,3.54,20-24
776,776,Iral74,21,Male,164,Exiled Doomblade,1.63,20-24
777,777,Yathecal72,20,Male,67,"Celeste, Incarnation of the Corrupted",3.46,20-24
778,778,Sisur91,7,Male,101,Final Critic,4.19,<10


In [42]:
purchase_count = purchase_data_df["Age Group"].value_counts()
purchase_count

20-24    365
15-19    136
25-29    101
30-34     73
35-39     41
10-14     28
<10       23
40+       13
Name: Age Group, dtype: int64

In [41]:
#total purchase value
purchase_data_group = purchase_data_df.groupby("Age Group")

total_purchase_value = purchase_data_group["Price"].sum()
total_purchase_value



Age Group
<10        77.13
10-14      82.78
15-19     412.89
20-24    1114.06
25-29     293.00
30-34     214.00
35-39     147.67
40+        38.24
Name: Price, dtype: float64

In [43]:
average_purchase_price = total_purchase_value / purchase_count
average_purchase_price

<10      3.353478
10-14    2.956429
15-19    3.035956
20-24    3.052219
25-29    2.900990
30-34    2.931507
35-39    3.601707
40+      2.941538
dtype: float64